<a href="https://colab.research.google.com/github/LuuZwide/Supervised-Learning/blob/main/Prediction_Model_using_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import yfinance as yf
drive.mount('/content/drive')
!cp "/content/drive/MyDrive/Functions/Preprocessing/Precossing_Functions_for_stock_chpters_1.ipynb" .
%run Precossing_Functions_for_stock_chpters_1.ipynb

from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC, LinearSVR, SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import confusion_matrix, roc_curve, precision_score, recall_score, precision_recall_curve,roc_auc_score
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, MaxAbsScaler

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
import tensorflow.keras.backend as K
from keras.activations import elu
from keras.engine.training import optimizer
from tensorflow.keras.callbacks import EarlyStopping
import math
import datetime

In [ ]:
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=700)
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())
ticker = yf.Ticker('IBM')
data = ticker.history(start=start_timestamp, end=end_timestamp, interval="1h")

data.reset_index(drop = False, inplace = True)
data = data.drop(['Dividends', 'Stock Splits'], axis = 1)
prices = data['Close']
stock_prices = prices
len_of_data = len(prices)
window_size = 5
stock_dataset = Combine_dataframes(data ,prices, window_size,len_of_data, fast = 12, slow = 26)
stock_dataset = stock_dataset.drop('next_close', axis = 1) # the next close price will be calculated with a neural net
chart = stock_dataset
chart = chart.reset_index(drop=True)

macd:  Index(['MACD', 'Signal', 'Histogram'], dtype='object')
Index(['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume', 'moving_avg',
       'exm_avg', 'STOCHk_1_3_3', 'STOCHd_1_3_3', 'MACD', 'Signal',
       'Histogram'],
      dtype='object')


In [ ]:
data_pipeline = Pipeline(
  (
    ('ConvertDataFrame', Add_Closing_Prices(5)),
  )
)
test = data_pipeline.fit_transform(chart)
x = test[['Prev_Close_1','Prev_Close_2','Prev_Close_3','Prev_Close_4', 'Prev_Close_5','Close']]

x['Next_close'] = x['Close'].shift(-1)

x = x.drop(x.index[-1])
x.head(-5)

<ipython-input-48-dae7d93b6451>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Next_close'] = x['Close'].shift(-1)


,Prev_Close_1,Prev_Close_2,Prev_Close_3,Prev_Close_4,Prev_Close_5,Close,Next_close
0,134.990433,134.521988,134.483749,134.412048,134.923416,134.732315,135.086044
1,134.732315,134.990433,134.521988,134.483749,134.412048,135.086044,135.066925
2,135.086044,134.732315,134.990433,134.521988,134.483749,135.066925,134.976105
3,135.066925,135.086044,134.732315,134.990433,134.521988,134.976105,135.124283
4,134.976105,135.066925,135.086044,134.732315,134.990433,135.124283,135.783936
...,...,...,...,...,...,...,...
3341,137.820007,137.804993,138.270004,137.970001,138.710007,137.720001,137.559998
3342,137.720001,137.820007,137.804993,138.270004,137.970001,137.559998,136.434998
3343,137.559998,137.720001,137.820007,137.804993,138.270004,136.434998,136.345001
3344,136.434998,137.559998,137.720001,137.820007,137.804993,136.345001,136.470001


In [ ]:
y = x['Next_close']
X = x.drop('Next_close',axis = 1)

In [ ]:
x.shape

(3351, 7)

In [ ]:
inputs = keras.layers.Input(shape=6)
preprocessing = keras.layers.Normalization()
preprocessing.adapt(X.values)

In [ ]:
preprocessed_inputs = preprocessing(X.values)
print(preprocessed_inputs)

tf.Tensor(
[[0.4206862  0.35561344 0.35038438 0.3405291  0.41172087 0.38486192]
 [0.3847995  0.4207441  0.35570112 0.35049894 0.34061363 0.4340426 ]
 [0.43397927 0.38485646 0.42083424 0.3558159  0.35058388 0.43138435]
 ...
 [0.6361123  0.6681685  0.6265552  0.60930055 0.6219108  0.6111669 ]
 [0.61110026 0.63617593 0.66826797 0.62668145 0.6093965  0.558333  ]
 [0.5582673  0.61116326 0.6362743  0.668396   0.6267782  0.22326064]], shape=(3351, 6), dtype=float32)


In [ ]:
for i in range(len(X.values)):
  print(preprocessing(X.values[i]))
  preprocessing.adapt(X.values[i])

tf.Tensor([[0.4206862  0.35561344 0.35038438 0.3405291  0.41172087 0.38486192]], shape=(1, 6), dtype=float32)
tf.Tensor([[ 0.24860327  1.415466   -0.7022146  -0.875078   -1.1992142   1.8476934 ]], shape=(1, 6), dtype=float32)
tf.Tensor([[ 1.4860102   0.10855854  1.1136906  -0.7104732  -0.85937726  1.4115582 ]], shape=(1, 6), dtype=float32)
tf.Tensor([[ 1.0200315   1.0970116  -0.3272112   0.71204996 -1.174053    0.654361  ]], shape=(1, 6), dtype=float32)
tf.Tensor([[ 0.3963561   0.84366584  0.93783236 -0.8043609   0.46692464  1.1261654 ]], shape=(1, 6), dtype=float32)
tf.Tensor([[ 0.99577546 -0.15459041  0.550484    0.69891447 -2.0472271   6.1169233 ]], shape=(1, 6), dtype=float32)
tf.Tensor([[ 2.0473137  -0.01243539 -0.4751178  -0.19153365 -0.13183424  3.6294675 ]], shape=(1, 6), dtype=float32)
tf.Tensor([[ 1.8653433  0.8182398 -0.5449466 -0.8511599 -0.6634777  2.1419158]], shape=(1, 6), dtype=float32)
tf.Tensor([[ 1.3783847   1.1515973   0.2929801  -0.82482266 -1.0759152   1.2163936 ]

KeyboardInterrupt: ignored

In [ ]:
lr = 0.001
dropout_rate = 0.15


model = keras.Sequential([
    keras.layers.Dense(units = 256, activation = 'relu', use_bias = True, kernel_initializer = 'glorot_uniform' ),
    keras.layers.Dense(units = 124, activation = 'relu',kernel_initializer = 'glorot_uniform'),
    keras.layers.Dense(units = 64, activation = 'relu',kernel_initializer = 'glorot_uniform'),
    keras.layers.Dense(units = 32, activation = 'relu',kernel_initializer = 'glorot_uniform'),
    keras.layers.Dense(units = 16, activation = 'relu',kernel_initializer = 'glorot_uniform'),
    keras.layers.Dense(units = 1)
])


model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = lr),
    loss = tf.losses.MeanAbsoluteError(reduction = 'auto', name = 'mean_absolute_error'),
    metrics = ['mean_squared_error']
)

early_stopping = EarlyStopping(monitor = 'mean_squared_error', patience = 5)
#callbacks = [LearningRateScheduler(exponential_decay),early_stopping ]

history = model.fit(
                    x = X.values,
                    y = [y],
                    batch_size = 128,
                    epochs = 850,
                    #callbacks = [early_stopping]
                  )

In [ ]:
mse = model.evaluate(X.values, y, verbose = 0)
print('mse: ', mse)
rmse = np.sqrt(mse)
print('rmse: ', rmse)

mse:  [0.7315839529037476, 0.9228886961936951]
rmse:  [0.85532681 0.96067096]


In [ ]:
predictions = model.predict(X.values[-50:])

for xt,yt in zip(predictions, y[-50:]):
  print(xt, '  ',yt)

2/2 [==============================] - 0s 7ms/step
[135.85237]    135.389892578125
[136.01773]    135.25999450683594
[135.9353]    135.27000427246094
[135.88487]    135.32989501953125
[135.913]    135.3699951171875
[135.94263]    135.32000732421875
[135.92964]    136.2449951171875
[136.54144]    136.16000366210938
[136.74887]    136.03500366210938
[136.69962]    135.99000549316406
[136.62297]    135.9149932861328
[136.54446]    136.375
[136.81677]    136.4199981689453
[136.97833]    136.6529998779297
[137.18077]    138.1199951171875
[138.23729]    137.92999267578125
[138.54785]    137.885498046875
[138.55058]    137.58009338378906
[138.31006]    137.58880615234375
[138.20917]    137.61000061035156
[138.18614]    138.16439819335938
[138.57037]    138.66000366210938
[139.06941]    138.7100067138672
[139.2876]    138.35000610351562
[139.08948]    137.35350036621094
[138.31052]    137.5449981689453
[138.1136]    137.24000549316406
[137.89288]    137.8300018310547
[138.22064]    138.6199951